# Проект создания LSTM (long short-term memory) модели для дополнения текстов

#### Импорты основных библиотек (venv, файл requirements.txt)

In [1]:
import sys
import yaml
import torch
from torch import nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel
torch.backends.cudnn.benchmark = True

/media/all-c/U/Dophamine/Python/text_completion_lstm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Добавление в path локальных файлов из src для возможности вызова функций оттуда

In [2]:
sys.path.append('.')
sys.path.append('src')

In [3]:
from data_preprocess import create_train_val_test_dataloaders_from_text_file
from common_utils import download_file, setup_logging
from lstm_model import LSTMNextTokenPredictor
from train import train_code_completion_model
from GPT_eval import validate_pretrained_gpt2_model

In [17]:
with open('configs/config.yaml', 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)

print(config.keys())

dict_keys(['data', 'model', 'vocabulary', 'training', 'generation', 'evaluation', 'logging', 'checkpointing', 'transformer'])


#### Все стадии логгируются в папку /logs

## Скачиваем датасет, а затем его делим 80% - 10% - 10% (train - val - test)

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained(
    config["transformer"]["model_name"],
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)

tokenizer.pad_token = config["vocabulary"]["special_tokens"][0]
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token

In [6]:
download_file(config["data"]["url"], "./" + config["data"]["path"])

Файл ./data/tweets.txt уже существует, пропускаем загрузку


'./data/tweets.txt'

### Загружаем данные в DataLoader'ы (также создается отдельный очищенный txt файл, если есть, то повторно не создается)

In [7]:
train_loader, val_loader, test_loader = create_train_val_test_dataloaders_from_text_file(
    file_path_to_text_data=config["data"]["path"],
    tokenizer=tokenizer,
    maximum_sequence_length=config["model"]["max_sequence_length"],
    minimum_sequence_length=config["model"]["min_sequence_length"],
    batch_size_for_training=config["training"]["batch_size"],
    batch_size_for_validation=config["training"]["batch_size"],
    batch_size_for_testing=config["training"]["batch_size"],
    train_split_ratio=0.8,
    validation_split_ratio=0.1,
    test_split_ratio=0.1,
    number_of_dataloader_workers=4,
    random_seed_for_split=42,
    shuffle_training_data=True,
    max_rows_all=50000
)


Train samples: 40000 (80.0%)
Validation samples: 5000 (10.0%)
Test samples: 5000 (10.0%)


### Проверка DataLoader'ов (из-за неправильного заполнения батчей они у меня были только из <EOS> токенов)

In [8]:

# ==================== ИНСПЕКЦИЯ ДАТАСЕТА ====================
print("\n" + "="*80)
print("ИНСПЕКЦИЯ ВАЛИДАЦИОННОГО ДАТАСЕТА (первые 10 записей)")
print("="*80 + "\n")

samples_to_inspect = 10
samples_inspected = 0

for batch_index, batch_data in enumerate(val_loader):
    if samples_inspected >= samples_to_inspect:
        break

    input_ids_batch = batch_data['input_ids']
    attention_mask_batch = batch_data['attention_mask']
    labels_batch = batch_data['labels']

    actual_sequence_length = attention_mask_batch.sum(dim=1)
    batch_size = input_ids_batch.size(0)

    print(f"Батч {batch_index + 1}:")
    print(f"  Размер батча: {batch_size}")
    print(f"  Форма input_ids: {input_ids_batch.shape}")
    print(f"  Форма attention_mask: {attention_mask_batch.shape}")
    print(f"  Форма labels: {labels_batch.shape}")
    print()

    for sample_index in range(batch_size):
        if samples_inspected >= samples_to_inspect:
            break

        actual_length = int(actual_sequence_length[sample_index].item())
        context_length = actual_length * 3 // 4
        target_length = actual_length - context_length

        # Декодирование полной последовательности
        full_text = tokenizer.decode(
            input_ids_batch[sample_index, :actual_length].tolist(),
            skip_special_tokens=True
        )

        # Декодирование контекста
        context_text = tokenizer.decode(
            input_ids_batch[sample_index, :context_length].tolist(),
            skip_special_tokens=True
        )

        # Декодирование референса (целевой части)
        reference_text = tokenizer.decode(
            input_ids_batch[sample_index, context_length:actual_length].tolist(),
            skip_special_tokens=True
        )

        print(f"  Сэмпл {samples_inspected + 1}:")
        print(f"    Актуальная длина: {actual_length} токенов")
        print(f"    Длина контекста (75%): {context_length} токенов")
        print(f"    Длина цели (25%): {target_length} токенов")
        print(f"    Полный текст: '{full_text[:200]}{'...' if len(full_text) > 200 else ''}'")
        print(f"    Контекст: '{context_text[:150]}{'...' if len(context_text) > 150 else ''}'")
        print(f"    Референс: '{reference_text[:150]}{'...' if len(reference_text) > 150 else ''}'")
        print(f"    Количество уникальных токенов в сэмпле: {len(set(input_ids_batch[sample_index, :actual_length].tolist()))}")
        print()

        samples_inspected += 1

    print("-" * 80 + "\n")

print("="*80)
print("КОНЕЦ ИНСПЕКЦИИ")
print("="*80 + "\n")


ИНСПЕКЦИЯ ВАЛИДАЦИОННОГО ДАТАСЕТА (первые 10 записей)

Батч 1:
  Размер батча: 16
  Форма input_ids: torch.Size([16, 35])
  Форма attention_mask: torch.Size([16, 35])
  Форма labels: torch.Size([16, 35])

  Сэмпл 1:
    Актуальная длина: 26 токенов
    Длина контекста (75%): 19 токенов
    Длина цели (25%): 7 токенов
    Полный текст: 'is sorry she doesnt tweet as often as others :p but i do have a'
    Контекст: 'is sorry she doesnt tweet as often as others'
    Референс: ' :p but i do have a'
    Количество уникальных токенов в сэмпле: 16

  Сэмпл 2:
    Актуальная длина: 29 токенов
    Длина контекста (75%): 21 токенов
    Длина цели (25%): 8 токенов
    Полный текст: 'and now too many things have changed and i have nothing to say to them they left like the others they always'
    Контекст: 'and now too many things have changed and i have nothing to say to'
    Референс: ' them they left like the others they always'
    Количество уникальных токенов в сэмпле: 20

  Сэмпл 3:
    Акт

## После чего создаем нашу модель

In [9]:
model = LSTMNextTokenPredictor(tokenizer.vocab_size, embedding_dim=config["model"]["embedding_dim"], hidden_dim=config["model"]["hidden_dim"], num_layers=config["model"]["num_layers"], dropout=config["model"]["dropout"])
optimizer = torch.optim.Adam(model.parameters(), lr=config["training"]["learning_rate"])
criterion = nn.CrossEntropyLoss()

#### Скомпилируем модель для повышения производительности (у меня на ROCm разница 3-5 процентов)

In [10]:
try:
    model = torch.compile(model, mode='max-autotune')
    print("✅ Модель скомпилирована с torch.compile()")
except Exception as e:
    print(f"⚠️ torch.compile() не поддерживается: {e}")

✅ Модель скомпилирована с torch.compile()


### Примеры предсказаний пишутся в логи (train.log)

#### Старый датасет, без очистки

In [11]:
#trained_model = train_code_completion_model(model=model, train_loader=train_loader, val_loader=val_loader, tokenizer=tokenizer, n_epochs=10, lr=0.001, device="cuda" if torch.cuda.is_available() else "cpu")

#### Датасет с очисткой от @username, ссылок и тд.

In [12]:
trained_model = train_code_completion_model(model=model, train_loader=train_loader, val_loader=val_loader, tokenizer=tokenizer, n_epochs=config["training"]["num_epochs"], lr=config["training"]["learning_rate"], device="cuda" if torch.cuda.is_available() else "cpu")

Epoch 1/10 [Val]: 100%|██████████| 313/313 [00:03<00:00, 104.24it/s]


Epoch 1/10 | Train Loss: 6.8674 | Val Loss: 6.7421



Epoch 2/10 [ROUGE]:   2%|▏         | 5/313 [00:00<00:32,  9.43it/s]


Epoch 2/10 | Train Loss: 6.7438 | Val Loss: 6.7406 | ROUGE-1: 0.0259 | ROUGE-2: 0.0000



Epoch 3/10 [Val]: 100%|██████████| 313/313 [00:03<00:00, 101.51it/s]


...

Epoch 8/10 | Train Loss: 6.7101 | Val Loss: 6.7505 | ROUGE-1: 0.0300 | ROUGE-2: 0.0000



Epoch 9/10 [Val]: 100%|██████████| 313/313 [00:03<00:00, 101.56it/s]


Epoch 9/10 | Train Loss: 6.7088 | Val Loss: 6.7548



Epoch 10/10 [ROUGE]:   2%|▏         | 5/313 [00:00<00:30, 10.01it/s]


Epoch 10/10 | Train Loss: 6.7065 | Val Loss: 6.7536 | ROUGE-1: 0.0227 | ROUGE-2: 0.0000



#### Примеры предсказаний слов (взяты из лога)
```
2025-11-23 22:07:54 - train - INFO - Примеры автодополнения после эпохи 10/10:
2025-11-23 22:07:54 - train - INFO - ================================================================================

2025-11-23 22:07:55 - train - INFO - Пример 1:
2025-11-23 22:07:55 - train - INFO - Контекст: is sorry she doesnt tweet as often as others
2025-11-23 22:07:55 - train - INFO - Предсказание: retgedesand that toldward
2025-11-23 22:07:55 - train - INFO - Референс:  :p but i do have a
2025-11-23 22:07:55 - train - INFO - --------------------------------------------------------------------------------

2025-11-23 22:07:55 - train - INFO - Пример 2:
2025-11-23 22:07:55 - train - INFO - Контекст: and now too many things have changed and i have nothing to say to
2025-11-23 22:07:55 - train - INFO - Предсказание:  like about ihh even thisburg sleep
2025-11-23 22:07:55 - train - INFO - Референс:  them they left like the others they always
2025-11-23 22:07:55 - train - INFO - --------------------------------------------------------------------------------

2025-11-23 22:07:55 - train - INFO - Пример 3:
2025-11-23 22:07:55 - train - INFO - Контекст: i hope today is a good one. on
2025-11-23 22:07:55 - train - INFO - Предсказание:  livingw, the awesome seen
2025-11-23 22:07:55 - train - INFO - Референс: l six days left till schools out
2025-11-23 22:07:55 - train - INFO - --------------------------------------------------------------------------------

2025-11-23 22:07:55 - train - INFO - Пример 4:
2025-11-23 22:07:55 - train - INFO - Контекст: tmj is this craziness with your jaw and it makes your jaw pop and it hurts and as far
2025-11-23 22:07:55 - train - INFO - Предсказание:  their think in for i't pent my watching
2025-11-23 22:07:55 - train - INFO - Референс:  as i know i'm still going to n
2025-11-23 22:07:55 - train - INFO - --------------------------------------------------------------------------------

2025-11-23 22:07:55 - train - INFO - Пример 5:
2025-11-23 22:07:55 - train - INFO - Контекст: 
2025-11-23 22:07:55 - train - INFO - Предсказание: i we and guide he
2025-11-23 22:07:55 - train - INFO - Референс: think you may wait
2025-11-23 22:07:55 - train - INFO - --------------------------------------------------------------------------------
```

### Сохраняем модель в формате pth (файл, используемый библиотекой PyTorch, который сохраняет состояние обученной нейронной сети, включая её параметры (веса и смещения))

In [18]:
torch.save(model.state_dict(), config["model"]["save_path"])

### Запуск валидации предобученной модели (distilgpt2)

In [14]:
# загрузка модели с HF (нужен интернет)
gpt2_model = GPT2LMHeadModel.from_pretrained(config["transformer"]["model_name"])

'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: 8be25cb3-f629-4f80-abfc-684224bab0ad)')' thrown while requesting HEAD https://huggingface.co/distilgpt2/resolve/main/config.json
Retrying in 1s [Retry 1/5].


In [16]:
# установка токена окончания последовательности
tokenizer.pad_token = tokenizer.eos_token

# запуск самой валидации
validation_results = validate_pretrained_gpt2_model(
    tokenizer=tokenizer,
    model=gpt2_model,
    validation_dataloader=val_loader,
    device="cuda" if torch.cuda.is_available() else "cpu",
    num_prediction_samples=5,
    max_generation_length=50,
    calculate_rouge_metrics=True
)

# Вывод результатов
print(f"\nИтоговые метрики:")
print(f"Validation Loss: {validation_results['validation_loss']:.4f}")
print(f"ROUGE-1: {validation_results['rouge1']:.4f}")
print(f"ROUGE-2: {validation_results['rouge2']:.4f}")

ROUGE calculation: 100%|██████████| 313/313 [01:01<00:00,  5.10it/s, R1=0.0935, R2=0.0222]


РЕЗУЛЬТАТЫ ВАЛИДАЦИИ ПРЕДОБУЧЕННОЙ МОДЕЛИ
Validation Loss: 6.3300
ROUGE-1: 0.0935
ROUGE-2: 0.0222


Итоговые метрики:
Validation Loss: 6.3300
ROUGE-1: 0.0935
ROUGE-2: 0.0222


## Выводы

### Выводы по сравнению моделей

Как я и предполагал, предобученная на качественных данных, еще и в большем количестве, GPT-2 показала лучшие результаты по ROUGE, хоть и все равно довольно низкие, но в три раза почти что выше. Также я уменьшил вчетверо размер всех выборок, так как не заметил при тестах (ячейки уже не сохранились, мешали своим выводом ориентироваться в тетради) значимых изменений с примерно 20000 и до конца. Все тренировки и инфернесы я проводил локально на RX 9070XT. По этой причине батчи я выставил в 16 единиц.

У меня стабильно получаются train_loss и val_loss примероно равными, при этом они высоки. Это означает, что модель недообучается. Ей не хватает сложности, при этом она нетребовательна к ресурсам тренировки, валидации и инференса.

#### Как улучшить?

Для улучшения я вижу два пути:
- Увеличить количество слоев. Это позволит лучше запоминать цепочки признаков и лучше генерировать ответ.
- Увеличить количество эпох, изменить параметры модели, изменить активацию и тд.
- Увеличить количество параметров. Не уверен, что это поможет лучше понимать признаки, но должно помочь.

### Бэкстейдж и проблемы

При работе я столкнулся с несколькими сложностями:
- При инициализации даталодеров я случайно заполнил ВСЕ батчи ```<EOS>``` токенами. Это вызвало десятикратное падение скорости обучения (с 30+ до 3.5), но выдавало очень низкую ошибку как на тренировочных, так и на валидационных данных.
- Ошибка с положением паддингов в токенайзере. Я пытался побороть ее самыми разными способами (за кадром, также не оставил в итоговый вариант тетради), но ошибка все никак не уходила. Решил посмотреть, как будет генерироваться, и, вроде бы, более-менее осмысленно получилось.